In [ ]:
#| default_exp configuration

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export

from typing import Literal
from typing_extensions import Self
from pathlib import Path
from importlib.metadata import version
from uuid import uuid4
from glob import glob

from pydantic import Field, BaseModel, model_validator
from pydantic_settings import (
    BaseSettings,
    SettingsConfigDict,
    CliMutuallyExclusiveGroup,
    TomlConfigSettingsSource,
    CliApp,
    CliSuppress
)

import clip_plot # for version

In [ ]:
#| export

class Paths(BaseModel):
    image_glob: str | None = Field(None, description="Glob of images")
    image_dir: Path | None = Field(None, description="Path to image folder")
    images: CliSuppress[list[Path]] = Field([])
    table_glob: list[Path] | None = Field(None,
                               description="Glob of table(s) with image_path, embed_path cols")
    table_dir: Path | None = Field(None, description="Folder with table(s) with image_path, embed_path cols")
    tables: CliSuppress[list[Path] | None] = Field([])
    meta_glob: list[Path] | None = Field(None,
                               description="Glob of table(s) with image_path, embed_path cols")
    meta_dir: Path | None = Field(None, description="Folder with table(s) with image_path, embed_path cols")
    metadata: CliSuppress[list[Path] | None] = Field([], description="Path(s) to csv or JSON table with metadata")
    table_id: str = Field(default_factory=lambda: str(uuid4()), description="identifier for table output")
    output_dir: Path = Field((Path()/"clipplot_output").resolve(),
            description="Directory for output data files and viewer")
    table_format: Literal["parquet", "csv"] = Field("parquet", description="Format for table, `csv` or `parquet`")

    @model_validator(mode='after')
    def check_glob_or_folder(self) -> Self:
        if self.image_glob is not None and self.image_dir is not None:
            raise ValueError("'image_glob' and 'image_dir' are mutually exclusive.")
        if self.table_glob is not None and self.table_dir is not None:
            raise ValueError("'tables_glob' and 'tables_dir' are mutually exclusive.")
        if self.meta_glob is not None and self.meta_dir is not None:
            raise ValueError("'meta_glob' and 'meta_dir' are mutually exclusive.")
        return self
    
    @model_validator(mode="after")
    def expand_globs(self) -> Self:
        if self.image_glob and not self.image_dir:
            self.images = [Path(p) for p in glob(self.image_glob, recursive=True)]
        return self

    # @model_validator(mode='after')
    # def check_table_vs_meta(self) -> Self:
    #     if self.metadata is not None and self.tables is not None:
    #         raise ValueError("'metadata' and 'tables' are mutually exclusive.")
    #     return self

class UmapSpec(BaseModel):
    n_neighbors: list[int] = Field([15], description="Number of neighbors in UMAP")
    min_dist: list[float] = Field([0.1], description="Minimum distance in UMAP")
    metric: CliSuppress[str] = Field("correlation")
    umap_on_full_dims: CliSuppress[bool] = Field(True)

class ClusterSpec(BaseModel):
    n_clusters: CliSuppress[int] = Field(12)
    max_clusters: CliSuppress[int] = Field(10)
    min_cluster_size: CliSuppress[int] = Field(20)
    cluster_preproc_dims: CliSuppress[int] = Field(-1)

class Cfg(BaseSettings):
    thumbnail_size: int = Field(128, description="Size of images in main bedmap view")
    model: str = Field("timm/convnext_tiny.dinov3_lvd1689m",
                            description="Model name on huggingface.co/models")
    umap_spec: UmapSpec = UmapSpec()
    clipplot_version: str = Field(version(clip_plot.__name__), description="Version of clipplot")
    plot_id: str = Field(default_factory=lambda: str(uuid4()), description="Unique identifier for plot")
    paths: Paths = Paths()

    # excluded from CLI - it's a hairball
    seed: CliSuppress[int] = Field(42)
    geojson: CliSuppress[None | Path] = Field(None)
    shuffle: CliSuppress[None | bool] = Field(False)
    copy_web_only: CliSuppress[bool] = Field(False)
    use_cache: CliSuppress[bool] = Field(False)
    encoding: CliSuppress[str] = Field("utf8")
    pointgrid_fill: CliSuppress[float] = Field(0.05)
    cell_size: CliSuppress[int] = Field(32)
    lod_cell_height: CliSuppress[int] = Field(128)
    min_size: CliSuppress[int] = Field(100, description="min edge for image")
    gzip: CliSuppress[bool] = Field(False)
    logo: CliSuppress[None | Path] = Field(None)
    tagline: CliSuppress[None] | str = Field(None)


    model_config = SettingsConfigDict(
        env_prefix = "CLIPPLOT_",
        cli_parse_args = True,
        use_attribute_docstrings = True,
        cli_prog_name = "clipplot",
        cli_hide_none_type = True,
        cli_ignore_unknown_args=True,
        # pyproject_toml_table_header=(),
    )

In [ ]:
#| hide

cfg = Cfg()

In [ ]:
old_version = cfg.model_dump()
old_version

{'thumbnail_size': 128,
 'model': 'timm/convnext_tiny.dinov3_lvd1689m',
 'umap_spec': {'n_neighbors': [15],
  'min_dist': [0.1],
  'metric': 'correlation',
  'umap_on_full_dims': True},
 'clipplot_version': '0.1.1a9',
 'plot_id': '4f1ea363-aff9-4298-8699-8b7a57d89a75',
 'paths': {'image_glob': None,
  'image_dir': None,
  'images': [],
  'table_glob': None,
  'table_dir': None,
  'tables': [],
  'meta_glob': None,
  'meta_dir': None,
  'metadata': [],
  'table_id': '877769e5-92b7-474d-ae80-a753f4e6b750',
  'output_dir': Path('/home/willsa/git/clip-plot/nbs/clipplot_output'),
  'table_format': 'parquet'},
 'seed': 42,
 'geojson': None,
 'shuffle': False,
 'copy_web_only': False,
 'use_cache': False,
 'encoding': 'utf8',
 'pointgrid_fill': 0.05,
 'cell_size': 32,
 'lod_cell_height': 128,
 'min_size': 100,
 'gzip': False,
 'logo': None,
 'tagline': None}

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()